<img src="../artworks/matchzoo-logo.png" alt="logo" style="width:600px;float: center"/>

# MatchZoo Quick Start

In [1]:
import matchzoo as mz
print(mz.__version__)

Using TensorFlow backend.


2.1.0


## Define Task

There are two types of tasks available in MatchZoo. `mz.tasks.Ranking` and `mz.tasks.Classification`. We will use a ranking task for this demo.

In [2]:
task = mz.tasks.Ranking()
print(task)

Ranking Task


## Prepare Data

In [3]:
train_raw = mz.datasets.toy.load_data(stage='train', task=task)
test_raw = mz.datasets.toy.load_data(stage='test', task=task)

In [4]:
type(train_raw)

matchzoo.data_pack.data_pack.DataPack

`DataPack` is a MatchZoo native data structure that most MatchZoo data handling processes build upon. A `DataPack` is consists of three `pandas.DataFrame`:

In [5]:
train_raw.left.head()

,text_left
id_left,
Q1,how are glacier caves formed?
Q2,How are the directions of the velocity and for...
Q5,how did apollo creed die
Q6,how long is the term for federal judges
Q7,how a beretta model 21 pistols magazines works


In [6]:
train_raw.right.head()

,text_right
id_right,
D1-0,A partly submerged glacier cave on Perito More...
D1-1,The ice facade is approximately 60 m high
D1-2,Ice formations in the Titlis glacier cave
D1-3,A glacier cave is a cave formed within the ice...
D1-4,"Glacier caves are often called ice caves , but..."


In [7]:
train_raw.relation.head()

,id_left,id_right,label
0,Q1,D1-0,0.0
1,Q1,D1-1,0.0
2,Q1,D1-2,0.0
3,Q1,D1-3,1.0
4,Q1,D1-4,0.0


It is also possible to convert a `DataPack` into a single `pandas.DataFrame` that holds all information.

In [8]:
train_raw.frame().head()

,id_left,text_left,id_right,text_right,label
0,Q1,how are glacier caves formed?,D1-0,A partly submerged glacier cave on Perito More...,0.0
1,Q1,how are glacier caves formed?,D1-1,The ice facade is approximately 60 m high,0.0
2,Q1,how are glacier caves formed?,D1-2,Ice formations in the Titlis glacier cave,0.0
3,Q1,how are glacier caves formed?,D1-3,A glacier cave is a cave formed within the ice...,1.0
4,Q1,how are glacier caves formed?,D1-4,"Glacier caves are often called ice caves , but...",0.0


However, using such `pandas.DataFrame` consumes much more memory if there are many duplicates in the texts, and that is the exact reason why we use `DataPack`. For more details about data handling, consult `matchzoo/tutorials/data_handling.ipynb`.

## Preprocessing

MatchZoo preprocessors are used to convert a raw `DataPack` into a `DataPack` that ready to be fed into a model. 

In [9]:
preprocessor = mz.preprocessors.BasicPreprocessor()

There are two steps to use a preprocessor. First, `fit`. Then, `transform`. `fit` will only changes the preprocessor's inner state but not the input `DataPack`.

In [10]:
preprocessor.fit(train_raw)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 13/13 [00:00<00:00, 887.44it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 100/100 [00:00<00:00, 4239.45it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 1665/1665 [00:00<00:00, 2126527.45it/s]


`fit` will gather useful information into its `context`, which will be used later in a `transform` or used to set hyper-parameters of a model.

In [11]:
preprocessor.context

{'filter_unit': <matchzoo.processor_units.processor_units.FrequencyFilterUnit at 0x12d085978>,
 'vocab_unit': <matchzoo.processor_units.processor_units.VocabularyUnit at 0x12d2cc9e8>,
 'vocab_size': 285,
 'embedding_input_dim': 285,
 'input_shapes': [(30,), (30,)]}

Once `fit`, the preprocessor has enough information to `transform`.  `transform` will not change the preprocessor's inner state and the input `DataPack`, but return a transformed `DataPack`.

In [12]:
train_processed = preprocessor.transform(train_raw)
test_processed = preprocessor.transform(test_raw)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 13/13 [00:00<00:00, 4162.92it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 100/100 [00:00<00:00, 4446.18it/s]
Processing text_right with transform: 100%|██████████| 100/100 [00:00<00:00, 65433.76it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 3/3 [00:00<00:00, 2686.36it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 20/20 [00:00<00:00, 5219.07it/s]
Processing text_right with transform: 100%|██████████| 20/20 [00:00<00:00, 37167.07it/s]


In [13]:
train_processed.left.head()

,text_left,length_left
id_left,,
Q1,"[152, 157, 7, 86, 51, 0, 0, 0, 0, 0, 0, 0, 0, ...",5
Q2,"[152, 157, 42, 144, 153, 42, 257, 195, 275, 26...",15
Q5,"[152, 193, 61, 176, 155, 0, 0, 0, 0, 0, 0, 0, ...",5
Q6,"[152, 12, 49, 42, 112, 88, 261, 111, 0, 0, 0, ...",8
Q7,"[152, 183, 215, 108, 147, 167, 73, 245, 0, 0, ...",8


As we can see, `text_left` is already in sequence form that nerual networks love.

Just to make sure we have the correct sequence:

In [14]:
vocab_unit = preprocessor.context['vocab_unit']
print('Orig Text:', train_processed.left.loc['Q1']['text_left'])
sequence = train_processed.left.loc['Q1']['text_left']
print('Transformed Indices:', sequence)
print('Transformed Indices Meaning:',
      '_'.join([vocab_unit.state['index_term'][i] for i in sequence]))

Orig Text: [152, 157, 7, 86, 51, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Transformed Indices: [152, 157, 7, 86, 51, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Transformed Indices Meaning: how_are_glacier_caves_formed_________________________


For more details about preprocessing, consult `matchzoo/tutorials/data_handling.ipynb`.

## Build Model

MatchZoo provides many built-in text matching models.

In [15]:
mz.models.list_available()

[matchzoo.models.naive.Naive,
 matchzoo.models.dssm.DSSM,
 matchzoo.models.cdssm.CDSSM,
 matchzoo.models.dense_baseline.DenseBaseline,
 matchzoo.models.arci.ArcI,
 matchzoo.models.arcii.ArcII,
 matchzoo.models.match_pyramid.MatchPyramid,
 matchzoo.models.knrm.KNRM,
 matchzoo.models.duet.DUET,
 matchzoo.models.drmmtks.DRMMTKS,
 matchzoo.models.drmm.DRMM,
 matchzoo.models.anmm.ANMM,
 matchzoo.models.mvlstm.MVLSTM,
 matchzoo.contrib.models.match_lstm.MatchLSTM]

Let's use `mz.models.DenseBaseline` for our demo.

In [16]:
model = mz.models.DenseBaseline()

The model is initialized with a hyper parameter table, in which values are partially filled. To view parameters and their values, use `print`.

In [17]:
print(model.params)

model_class                   <class 'matchzoo.models.dense_baseline.DenseBaseline'>
input_shapes                  None
task                          None
optimizer                     adam
with_multi_layer_perceptron   True
mlp_num_units                 256
mlp_num_layers                3
mlp_num_fan_out               64
mlp_activation_func           relu


`to_frame` gives you more informartion in addition to just names and values.

In [18]:
model.params.to_frame()[['Name', 'Description', 'Value']]

,Name,Description,Value
0,model_class,Model class. Used internally for save/load. Ch...,<class 'matchzoo.models.dense_baseline.DenseBa...
1,input_shapes,Dependent on the model and data. Should be set...,None
2,task,"Decides model output shape, loss, and metrics.",None
3,optimizer,None,adam
4,with_multi_layer_perceptron,A flag of whether a multiple layer perceptron ...,True
5,mlp_num_units,Number of units in first `mlp_num_layers` layers.,256
6,mlp_num_layers,Number of layers of the multiple layer percetron.,3
7,mlp_num_fan_out,Number of units of the layer that connects the...,64
8,mlp_activation_func,Activation function used in the multiple layer...,relu


To set a hyper-parameter:

In [19]:
model.params['task'] = task
model.params['mlp_num_units'] = 3
print(model.params)

model_class                   <class 'matchzoo.models.dense_baseline.DenseBaseline'>
input_shapes                  None
task                          Ranking Task
optimizer                     adam
with_multi_layer_perceptron   True
mlp_num_units                 3
mlp_num_layers                3
mlp_num_fan_out               64
mlp_activation_func           relu


Notice that we are still missing `input_shapes`, and that information is store in the preprocessor.

In [20]:
print(preprocessor.context['input_shapes'])

[(30,), (30,)]


We may use `update` to load a preprocessor's context into a model's hyper-parameter table.

In [21]:
model.params.update(preprocessor.context)

Now we have a completed hyper-parameter table.

In [22]:
model.params.completed()

True

With all parameters filled in, we can now build and compile the model.

In [23]:
model.build()
model.compile()

MatchZoo models are wrapped over keras models, and the `backend` property of a model gives you the actual keras model built.

In [24]:
model.backend.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 30)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 30)           0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 60)           0           text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 3)            183         concatenate_1[0][0]              
__________

For more details about models, consult `matchzoo/tutorials/models.ipynb`.

## Train, Evaluate, Predict

A `DataPack` can `unpack` itself into data that can be directly used to train a MatchZoo model.

In [25]:
x, y = train_processed.unpack()
test_x, test_y = test_processed.unpack()

In [26]:
model.fit(x, y, batch_size=32, epochs=5)

Epoch 1/5
100/100 [==============================] - 0s 3ms/step - loss: 76.6259
Epoch 2/5
100/100 [==============================] - 0s 66us/step - loss: 38.0670
Epoch 3/5
100/100 [==============================] - 0s 77us/step - loss: 16.3323
Epoch 4/5
100/100 [==============================] - 0s 63us/step - loss: 6.5714
Epoch 5/5
100/100 [==============================] - 0s 71us/step - loss: 2.7183


An alternative to train a model is to use a `DataGenerator`. This is useful for delaying expensive preprocessing steps or doing real-time data augmentation. For some models that needs dynamic batch-wise information, using a `DataGenerator` is required. For more details about `DataGenerator`, consult `matchzoo/tutorials/data_handling.ipynb`.

In [27]:
data_generator = mz.DataGenerator(train_processed, batch_size=32)

In [28]:
model.fit_generator(data_generator, epochs=5, use_multiprocessing=True, workers=4)

Epoch 1/5
4/4 [==============================] - 0s 19ms/step - loss: 0.6111
Epoch 2/5
4/4 [==============================] - 0s 28ms/step - loss: 0.2031
Epoch 3/5
4/4 [==============================] - 0s 32ms/step - loss: 0.0566
Epoch 4/5
4/4 [==============================] - 0s 30ms/step - loss: 0.1580
Epoch 5/5
4/4 [==============================] - 0s 35ms/step - loss: 0.1418


In [29]:
model.evaluate(test_x, test_y)

{mean_average_precision(0.0): 0.037037037037037035}

In [30]:
model.predict(test_x)

array([[-0.05505364],
       [-0.05505364],
       [-0.74016404],
       [-0.32844803],
       [-0.05505364],
       [-0.05505364],
       [-0.05505364],
       [-0.05505364],
       [-0.18103671],
       [-0.05505364],
       [-0.05505364],
       [-0.05505364],
       [-0.13401566],
       [-0.05505364],
       [-0.05505364],
       [-0.6933212 ],
       [-0.14404102],
       [-0.7520623 ],
       [-0.82780886],
       [-0.22631875]], dtype=float32)

## A Shortcut to Preprocessing and Model Building

Since data preprocessing and model building are laborious and special setups of some models makes this even worse, MatchZoo provides `prepare`, a unified interface that handles interaction among data, model, and preprocessor automatically.

More specifically, `prepare` does these following things:
 - create a default preprocessor of the model class (if not given one)
 - fit the preprocessor using the raw data
 - create an embedding matrix
 - instantiate a model and fill in hype-parameters
 - build the model
 - instantiate a `DataGeneratorBuilder` that will build a correctly formed `DataGenerator` given a `DataPack`
 
It also does many special handling for specific models, but we will not go into the details of that here.

In [31]:
for model_class in mz.models.list_available():
    print(model_class)
    model, preprocessor, data_generator_builder, embedding_matrix = mz.prepare(
        task=task,
        model_class=model_class,
        data_pack=train_raw,
    )
    train_processed = preprocessor.transform(train_raw, verbose=0)
    test_processed = preprocessor.transform(test_raw, verbose=0)
    train_gen = data_generator_builder.build(train_processed)
    test_gen = data_generator_builder.build(test_processed)
    model.fit_generator(train_gen, epochs=1)
    model.evaluate_generator(test_gen)
    print()

<class 'matchzoo.models.naive.Naive'>
Epoch 1/1
1/1 [==============================] - 0s 138ms/step - loss: 53812.7305

<class 'matchzoo.models.dssm.DSSM'>
Epoch 1/1
1/1 [==============================] - 1s 534ms/step - loss: 0.1385

<class 'matchzoo.models.cdssm.CDSSM'>
Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 0.0980

<class 'matchzoo.models.dense_baseline.DenseBaseline'>
Epoch 1/1
1/1 [==============================] - 0s 429ms/step - loss: 894.1340

<class 'matchzoo.models.arci.ArcI'>
Epoch 1/1
1/1 [==============================] - 1s 619ms/step - loss: 0.0550

<class 'matchzoo.models.arcii.ArcII'>
Epoch 1/1
1/1 [==============================] - 1s 722ms/step - loss: 0.0759

<class 'matchzoo.models.match_pyramid.MatchPyramid'>
Epoch 1/1
1/1 [==============================] - 1s 533ms/step - loss: 0.0610

<class 'matchzoo.models.knrm.KNRM'>
Epoch 1/1
1/1 [==============================] - 1s 942ms/step - loss: 1074.5272

<class 'matchzoo.models.duet.DUE

## Save and Load the Model

In [32]:
model.save('my-model')
loaded_model = mz.load_model('my-model')